## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime
import numpy as np



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [95]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    
                    
                if name in list(check_sheet.Workflow):
                    print(name)
                    dfy = check_sheet[check_sheet['Workflow']== name]
            
                    dfy.reset_index(drop=True)

                    for index,curie_id in enumerate(dfy.Curie):
                        print(index,curie_id)
                        node_num = dfy.iloc[index][3]
                        query_id = curie_id
                        if np.isnan(dfy.iloc[index][2]) == True:
                            len_check = len(child_response['fields']['data']['message']['results'])
                        else:
                            len_check = dfy.iloc[index][2]

                        #print(node_num, query_id, len_check)

                        locs = []
                        for x, val in enumerate(child_response['fields']['data']['message']['results']):
                            #print(val)

                            if x < len_check:

                                if query_id in val['node_bindings'][node_num][0]['id']:
                                    locs.append(x)
                        if not locs:
                            check_result = f'False'
                            print(check_result)
                            #pass
                        else:
                            check_result = f'True'
                            print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                        dict3[curie_id] = check_result    
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [96]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [97]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1a_DILI-three-hop-from-MONDO:0005359_DILI.json,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,CHEMBL.COMPOUND:CHEMBL1698267,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable;...
4,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,CHEMBL.COMPOUND:CHEMBL165,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,CHEMBL.COMPOUND:CHEMBL116438,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,B.1d_DILI_three-hop-from-SNOMEDCT:197354009_To...,CHEMBL.COMPOUND:CHEMBL1698267,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable;...
9,D.1_parkinsons-crohns.json,NCBIGene:120892,NaN,n01,include,NaN,LRRK2


<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [4]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule.json') or (name == 'C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule.json') or (name == 'C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(600)

                    result_status = retrieve_ars_results(kcresult)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=3e1840c3-5eae-438a-aea6-ce258f7ae57a
Done
Done
kp-genetics Done 0
Error
kp-cam Error 0
Done
kp-icees-dili Done 0
Error
ara-aragorn-exp Error 0
Done
ara-improving Done 5
Done
ara-aragorn Done 443
Done
kp-molecular Done 97
Error
ara-explanatory Error 0
Unknown
ara-ncats Unknown 0
Done
ara-arax Done 259
Done
kp-icees Done 0
Done
kp-textmining Done 0
Done
kp-chp Done 30
Done
kp-cohd ARS Error 0
Error
ara-robokop Error 0
Done
kp-openpredict Done 0
Done
ara-unsecret Done 100
Done
ara-bte Done 82
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=3821db63-7357-41a5-b1e0-2cc48618dde7
Done
Error
kp-cam Error 0
Done
kp-icees-dili ARS Error 0
Error
kp-cohd Error 0
Done
kp-textmining Done 0
Unknown
kp-genetics Unknown 0
Done
ara-improving ARS Error 0
Done
ara-unsecre

kp-icees-dili Error 0
Error
ara-arax Error 0
Error
kp-molecular Error 0
Error
ara-aragorn Error 0
Error
kp-cam Error 0
Error
ara-aragorn-exp Error 0
Error
kp-genetics Error 0
Done
kp-textmining Done 0
Error
kp-icees Error 0
Unknown
ara-ncats Unknown 0
Error
ara-robokop Error 0
Error
ara-improving Error 0
Done
kp-chp Done 0
Done
ara-unsecret Done 0
Error
kp-cohd Error 0
Done
ara-bte Done 0
Error
ara-explanatory Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone.json
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
https://arax.ncats.io/?source=ARS&id=050ad5e3-33d5-4484-97a1-36f220aec209
Done
Error
ara-unsecret Error 0
Error
ara-robokop Error 0
Error
kp-cam Error 0
Done
kp-molecular Done 295
Done
kp-textmining Done 0
Error
ara-explanatory Error 0
Done
kp-cohd Done 50
Error
ara-aragorn Error 0
Unknown
ara-ncats Unknown 0
Done
kp-chp Done 0
Done
ara-arax Done 500
Done
kp-genetics Done 0
Done
ara-bte D

https://arax.ncats.io/?source=ARS&id=940d9ed3-b57c-422f-a80b-da52b4823781
Done
Done
kp-genetics Done 0
Done
kp-icees Done 0
Done
kp-cohd ARS Error 0
Done
kp-textmining Done 0
Unknown
ara-arax Unknown 0
Error
ara-aragorn-exp Error 0
Error
ara-unsecret Error 0
Done
kp-icees-dili Done 0
Done
kp-molecular Done 0
Error
ara-bte Error 0
Error
ara-explanatory Error 0
Unknown
ara-ncats Unknown 0
Done
ara-aragorn Done 102
Done
ara-improving Done 10
Error
ara-robokop Error 0
Done
kp-chp Done 0
Done
kp-openpredict Done 0
Error
kp-cam Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone.json
B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
https://arax.ncats.io/?source=ARS&id=10f8b501-7f40-42b0-834f-a8ade5943c76
Done
Done
kp-textmining Done 0
Done
ara-improving Done 176
Error
ara-aragorn-exp Error 0
Error
kp-cam Error 0
Done
kp-chp Done 0
Done
kp-cohd Done 50
Error
ara-explanatory Error 0
Done
kp-i

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [84]:
dict_workflows

{'A.3_KCNMA1': '3e1840c3-5eae-438a-aea6-ce258f7ae57a',
 'A.2a_RHOBTB2_twohop': '3821db63-7357-41a5-b1e0-2cc48618dde7',
 'A.8_EGFR_simple': '0ffb4378-ba50-4e45-81ba-02dd137bb46d',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': '85e0cdf0-41f4-403e-b618-146953c92454',
 'A.0_RHOBTB2_direct': 'f5e0cbc7-dd83-4cf3-be3b-7cb05705402b',
 'A.2_RHOBTB2_twohop': '0f94dbfb-b821-4fc6-8e1a-955b81a61211',
 'A.Multiomics_BigGIM_DR_KP_EGFR': '6ed81cf3-84d2-4b9b-8455-b907c1a2c198',
 'A.2_RHOBTB2_twohop_constrained': '6d667261-ca76-4cc7-bf72-3035c76c011d',
 'A.9_EGFR_advanced': '9a4e6e57-6433-4bcd-a92f-386e8527eeea',
 'A.2a_expanded_RHOBTB2_twohop_constrained': '6e66d6ee-3423-4a82-9c38-d32eb7aa254f',
 'A.1_RHOBTB2': 'c78d2ea6-3f5a-4e60-8e14-7c87f094134e',
 'A.2a_expanded_RHOBTB2_twohop': 'aaeaae14-ca7f-4349-b8a6-aff423dbb06e',
 'B.4e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate': 'c0155a93-f792-4440-8a07-6c4c97494f29',
 'B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone': '050ad5e3-33d5-4484-97a1-36f2

In [98]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    
    result_status = retrieve_ars_results(val, name, check_sheet)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1.json 3e1840c3-5eae-438a-aea6-ce258f7ae57a
Done
Done
kp-genetics Done 0
Error
kp-cam Error 0
Done
kp-icees-dili Done 0
Error
ara-aragorn-exp Error 0
Done
ara-improving Done 5
Done
ara-aragorn Done 443
Done
kp-molecular Done 97
Error
ara-explanatory Error 0
Unknown
ara-ncats Unknown 0
Done
ara-arax Done 259
Done
kp-icees Done 0
Done
kp-textmining Done 0
Done
kp-chp Done 30
Done
kp-cohd ARS Error 0
Error
ara-robokop Error 0
Done
kp-openpredict Done 0
Done
ara-unsecret Done 100
Done
ara-bte Done 82
A.2a_RHOBTB2_twohop.json 3821db63-7357-41a5-b1e0-2cc48618dde7
Done
Error
kp-cam Error 0
Done
kp-icees-dili ARS Error 0
Error
kp-cohd Error 0
Done
kp-textmining Done 0
Unknown
kp-genetics Unknown 0
Done
ara-improving ARS Error 0
Done
ara-unsecret Done 0
Done
ara-bte Done 0
Done
ara-aragorn Done 0
Done
kp-openpredict Done 0
Error
ara-explanatory Error 0
Error
kp-molecular Error 0
Done
ara-arax Done 0
Done
kp-chp Done 0
Done
kp-icees ARS Error 0
Error
ara-aragorn-exp Error 0
Error
ara-ro

Done
Error
ara-bte Error 0
Error
kp-molecular Error 0
Error
ara-arax Error 0
Unknown
kp-genetics Unknown 0
Unknown
ara-ncats Unknown 0
Error
kp-cohd Error 0
Error
kp-cam Error 0
Error
ara-robokop Error 0
Done
kp-icees ARS Error 0
Error
ara-aragorn Error 0
Done
kp-icees-dili ARS Error 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
ara-improving ARS Error 0
Error
ara-unsecret Error 0
Error
ara-explanatory Error 0
Done
kp-chp Done 0
Done
kp-textmining Done 0
B.5_DILI_branched-four-hop.json 1306f06a-193c-4e98-a201-c0032f7e4940
Done
Done
ara-improving ARS Error 0
Done
kp-openpredict Done 0
Done
kp-icees-dili ARS Error 0
Done
kp-textmining Done 0
Error
ara-robokop Error 0
Error
kp-cam Error 0
Error
ara-aragorn-exp Error 0
Error
ara-explanatory Error 0
Error
ara-unsecret Error 0
Done
kp-icees ARS Error 0
Unknown
ara-ncats Unknown 0
Error
kp-cohd Error 0
Done
kp-chp Done 0
Error
ara-aragorn Error 0
Error
ara-bte Error 0
Error
ara-arax Error 0
Unknown
kp-genetics Unknown 0
Erro

ara-robokop Error 0
Done
kp-icees Done 0
Done
kp-chp Done 0
Unknown
ara-ncats Unknown 0
D.1_parkinsons-crohns.json abb8f85a-a0cc-4083-aad9-6695cb4b638a
Done
Error
ara-bte Error 0
Error
kp-cam Error 0
Error
ara-unsecret Error 0
Unknown
ara-arax Unknown 0
Unknown
ara-ncats Unknown 0
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
False
1 NCBIGene:11315
False
2 NCBIGene:110357
False
kp-chp Done 0
Error
kp-icees-dili Error 0
Done
kp-icees ARS Error 0
Error
kp-cohd Error 0
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
False
1 NCBIGene:11315
False
2 NCBIGene:110357
False
kp-openpredict Done 0
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
curie id: NCBIGene:120892 : INCLUDED at postion N == [0] on n01
1 NCBIGene:11315
curie id: NCBIGene:11315 : INCLUDED at postion N == [2] on n01
2 NCBIGene:110357
False
ara-improving Done 22
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
False
1 NCBIGene:11315
False
2 NCBIGene:110357
False
kp-textmining Done 0
Unknown
kp-genetics Unknown 0
Error


In [99]:
workflow_result_messages

{'A.3_KCNMA1': [{'pk_id': 'https://arax.ncats.io/?source=ARS&id=3e1840c3-5eae-438a-aea6-ce258f7ae57a',
   'kp-genetics': 'No Results: 200',
   'kp-cam': 'Error: 502',
   'kp-icees-dili': 'No Results: 200',
   'ara-aragorn-exp': 'Error: 404',
   'ara-improving': 'Results: 200{}',
   'ara-aragorn': 'Results: 200{}',
   'kp-molecular': 'Results: 200{}',
   'ara-explanatory': 'Error: 500',
   'ara-ncats': 'Unknown: 503',
   'ara-arax': 'Results: 200{}',
   'kp-icees': 'No Results: 200',
   'kp-textmining': 'No Results: 200',
   'kp-chp': 'Results: 200{}',
   'kp-cohd': 'ARS Error: 200',
   'ara-robokop': 'Error: 500',
   'kp-openpredict': 'No Results: 200',
   'ara-unsecret': 'Results: 200{}',
   'ara-bte': 'Results: 200{}'},
  {'ara-improving': {'infores:improving-agent',
    'infores:lincs',
    'infores:spoke'},
   'ara-aragorn': {'infores:aragorn',
    'infores:aragorn-ranker-ara',
    'infores:automat-ctd',
    'infores:automat-drug-central',
    'infores:automat-gtopdb',
    'infores

### Creating dataframe for workflows with PK

<br>

In [101]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)


for k in sorted(workflow_result_messages):
    print(k)
    
    if (k == 'C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule') or (k == 'C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule') or (k == 'C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule'):
            pass
    else:
    
        col.append(k)
    
    
    
    #count = 0
        for key, value in workflow_result_messages[k][0].items():
        
        
        #count= count+1
            final_dict[key].append(value)

    final_dict = dict(final_dict)
    #print(count)
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-related-to
B.2_DILI-three-hop-interacts-with
B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.4e_DILI-fourth-one-hop-

In [102]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [103]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=f5e0cbc7-...,https://arax.ncats.io/?source=ARS&id=c78d2ea6-...,https://arax.ncats.io/?source=ARS&id=0f94dbfb-...,https://arax.ncats.io/?source=ARS&id=6d667261-...,https://arax.ncats.io/?source=ARS&id=3821db63-...,https://arax.ncats.io/?source=ARS&id=aaeaae14-...,https://arax.ncats.io/?source=ARS&id=6e66d6ee-...,https://arax.ncats.io/?source=ARS&id=3e1840c3-...,https://arax.ncats.io/?source=ARS&id=0ffb4378-...,https://arax.ncats.io/?source=ARS&id=9a4e6e57-...,...,https://arax.ncats.io/?source=ARS&id=c7f343f0-...,https://arax.ncats.io/?source=ARS&id=10f8b501-...,https://arax.ncats.io/?source=ARS&id=1306f06a-...,https://arax.ncats.io/?source=ARS&id=29e84225-...,https://arax.ncats.io/?source=ARS&id=f5c18f63-...,https://arax.ncats.io/?source=ARS&id=abb8f85a-...,https://arax.ncats.io/?source=ARS&id=d680d232-...,https://arax.ncats.io/?source=ARS&id=883013f5-...,https://arax.ncats.io/?source=ARS&id=f87aa74f-...,https://arax.ncats.io/?source=ARS&id=e9549e6b-...
ara-arax,No Results: 200,Results: 200{},Results: 200{},Results: 200{},No Results: 200,Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Unknown: 503,Error: 598,Unknown: 503,Results: 200{},Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,No Results: 200,No Results: 200,Results: 200{},Error: 504,No Results: 200,No Results: 200,No Results: 200,Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Error: 504,Error: 504,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
kp-cohd,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,No Results: 200,Error: 400,Error: 400,...,No Results: 200,Results: 200{},Error: 400,Results: 200{},No Results: 200,Error: 400,Error: 400,No Results: 200,Error: 400,Error: 400
kp-icees-dili,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-aragorn,No Results: 200,Results: 200{},Error: 598,No Results: 200,No Results: 200,Results: 200{},No Results: 200,Results: 200{},Results: 200{},No Results: 200,...,Results: 200{},Error: 598,Error: 500,Results: 200{},No Results: 200,"Results: 200{'NCBIGene:120892': 'True', 'NCBIG...","Results: 200{'NCBIGene:3988': 'True', 'NCBIGen...",No Results: 200,Error: 500,Error: 598
kp-genetics,No Results: 200,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,Unknown: 503,No Results: 200,No Results: 200,Unknown: 503,Unknown: 503,No Results: 200,Unknown: 503,Unknown: 503
ara-bte,No 

### Creating second table with edge attribute source

<br>

In [14]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-related-to
B.2_DILI-three-hop-interacts-with
B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.4e_DILI-fourth-one-hop-

In [15]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.1_RHOBTB2': {'ara-arax': {'infores:arax', 'infores:rtx-kg2'},
              'ara-bte': {'infores:biothings-semmeddb-gene',
               'infores:semmeddb',
               'infores:translator-biothings-explorer'},
              'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-robokop',
               'infores:biothings-semmeddb-gene',
               'infores:hetio',
               'infores:lincs',
               'infores:rtx-kg2',
               'infores:semmeddb',
               'infores:spoke',
               'infores:sri-reference-kg',
               'infores:translator-biothings-explorer'},
              'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.2_RHOBTB2_tw

In [16]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [17]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-improving', 'infores:lincs'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:improving-agent'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:biothings-semmeddb-gene'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:translator-biothings-explorer'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:automat-robokop'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:biothings-semmeddb-gene'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:sri-reference-kg'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:hetio'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.1_RHOBTB2', 'ara-aragor

In [18]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [19]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [20]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [21]:
df2test = pd.DataFrame(df2test.unstack().T)

In [22]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN
JensenLab Knowledge UniProtKB-KW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN
JensenLab Text Mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-ontology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[kp-cohd],[kp-cohd],NaN,NaN,[kp-cohd],NaN,NaN
infores:sri-reference-kg,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,[ara-aragorn],"[ara-aragorn, ara-bte]",NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN
infores:string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving]


In [23]:
#df2test.drop([''], axis=0, inplace=True)

In [24]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [25]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN
JensenLab Knowledge UniProtKB-KW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN
JensenLab Text Mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-ontology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[kp-cohd],[kp-cohd],NaN,NaN,[kp-cohd],NaN,NaN
infores:sri-reference-kg,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,[ara-aragorn],"[ara-aragorn, ara-bte]",NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN
infores:string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving]


In [26]:
#df2.replace([], 'None', regex=True,inplace=True)

In [27]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [28]:
df2test = df2test.rename_axis(None)

In [29]:
df2test.columns.name = None

In [30]:
df2test.fillna('', inplace=True)

In [31]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
CTD,,,,,,,,,,,...,,,,,,[ara-aragorn],[ara-aragorn],,,
DisGeNET,,,,,,,,,,,...,,,,,,[ara-aragorn],[ara-aragorn],,,
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,
JensenLab Knowledge UniProtKB-KW,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,
JensenLab Text Mining,,,,,,,,,,,...,,,,,,[ara-aragorn],[ara-aragorn],,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-ontology,,,,,,,,,,,...,,,,[kp-cohd],[kp-cohd],,,[kp-cohd],,
infores:sri-reference-kg,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,[ara-aragorn],"[ara-aragorn, ara-bte]",,,,[ara-aragorn],[ara-aragorn],[ara-aragorn],,
infores:string,,,,,,,,,,,...,,,,,,,,,,[ara-improving]
infores:translator-biothings-explorer,,"[ara-bte, ara-aragorn]","[ara-bte, ara-arax]","[ara-bte, ara-arax]","[ara-bte, ara-aragorn, ara-arax]","[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-aragorn, ara-bte]","[ara-bte, ara-arax]","[ara-bte, ara-arax]",...,,[ara-bte],,,,,,,,


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [32]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [33]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]

In [34]:
infores_catalog = infores_catalog[:335]

In [35]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [36]:
dict_map

{'CTD': 'Illegal value',
 'DisGeNET': 'Illegal value',
 'JensenLab Knowledge GHR': 'Illegal value',
 'JensenLab Knowledge UniProtKB-KW': 'Illegal value',
 'JensenLab Text Mining': 'Illegal value',
 'Monarch': 'Illegal value',
 'UniProt Disease': 'Illegal value',
 'infores:aragorn': 'ARA',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat-biolink': 'KP',
 'infores:automat-covidkop': 'KP',
 'infores:automat-ctd': 'KP',
 'infores:automat-drug-central': 'KP',
 'infores:automat-gtopdb': 'KP',
 'infores:automat-hetio': 'KP',
 'infores:automat-hmdb': 'KP',
 'infores:automat-pharos': 'KP',
 'infores:automat-robokop': 'KP',
 'infores:automat-uberongraph': 'KP',
 'infores:bgee': 'External Source',
 'infores:bindingdb': 'External Source',
 'infores:biothings-dgidb': 'KP',
 'infores:biothings-multiomics-biggim-drugresponse': 'KP',
 'infores:biothings-semmeddb-gene': 'KP',
 'infores:chembl': 'External Source',
 'infores:civic': 'External Source',
 'infores:co

In [37]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [38]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog
CTD,,,,,,,,,,,...,,,,,[ara-aragorn],[ara-aragorn],,,,Illegal value
DisGeNET,,,,,,,,,,,...,,,,,[ara-aragorn],[ara-aragorn],,,,Illegal value
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,[ara-aragorn],,,,,Illegal value
JensenLab Knowledge UniProtKB-KW,,,,,,,,,,,...,,,,,[ara-aragorn],,,,,Illegal value
JensenLab Text Mining,,,,,,,,,,,...,,,,,[ara-aragorn],[ara-aragorn],,,,Illegal value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-ontology,,,,,,,,,,,...,,,[kp-cohd],[kp-cohd],,,[kp-cohd],,,KP
infores:sri-reference-kg,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,"[ara-aragorn, ara-bte]",,,,[ara-aragorn],[ara-aragorn],[ara-aragorn],,,KP
infores:string,,,,,,,,,,,...,,,,,,,,,[ara-improving],External Source
infores:translator-biothings-explorer,,"[ara-bte, ara-aragorn]","[ara-bte, ara-arax]","[ara-bte, ara-arax]","[ara-bte, ara-aragorn, ara-arax]","[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-aragorn, ara-bte]","[ara-bte, ara-arax]","[ara-bte, ara-arax]",...,[ara-bte],,,,,,,,,ARA


In [39]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [40]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:sri-ontology,,,,,,,,,,,...,,[kp-cohd],[kp-cohd],,,[kp-cohd],,,KP,KP
infores:lincs,[ara-improving],"[ara-aragorn, ara-improving]","[ara-bte, ara-improving]","[ara-improving, ara-bte]",[ara-aragorn],,[ara-improving],[ara-improving],[ara-improving],[ara-aragorn],...,,,,[ara-aragorn],"[ara-aragorn, ara-improving]",,,,KP,KP
infores:molepro,,,,,,,"[kp-molecular, ara-arax, ara-unsecret]","[kp-molecular, ara-unsecret]","[kp-molecular, ara-unsecret]","[ara-unsecret, kp-molecular]",...,[kp-molecular],,,,,,,,KP,KP
infores:mychem-info,,,[ara-arax],[ara-arax],,,[ara-aragorn],,,,...,,,,,,,,,KP,KP
infores:connections-hypothesis,,,[ara-arax],[ara-arax],,,"[ara-arax, kp-chp]",,,,...,,"[kp-chp, ara-aragorn]",,[ara-aragorn],,,,,KP,KP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:faers,,,,,,,,[ara-aragorn],,[ara-aragorn],...,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],,,Illegal value,NaN
infores:hetio,,[ara-aragorn],[ara-bte],[ara-bte],[ara-aragorn],,"[ara-aragorn, ara-arax, ara-bte]",,,[ara-aragorn],...,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],,,Illegal value,NaN
infores:molepro-kp,,,,,,,"[kp-molecular, ara-arax, ara-unsecret]","[kp-molecular, ara-unsecret]","[kp-molecular, ara-unsecret]","[ara-unsecret, kp-molecular]",...,,,,,,,,,Illegal value,NaN
infores:ontological-hierarchy,,,,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,,...,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],,,Illegal value,NaN


In [41]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [42]:
df2test['Query Type'] = 'Sync'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

<br>

## Converting the Pk's to hyperlink

<br>

In [104]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [105]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [106]:
df.sort_index(inplace=True)

In [107]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200{},Error: 598,No Results: 200,No Results: 200,Results: 200{},No Results: 200,Results: 200{},Results: 200{},No Results: 200,...,Results: 200{},Error: 598,Error: 500,Results: 200{},No Results: 200,"Results: 200{'NCBIGene:120892': 'True', 'NCBIG...","Results: 200{'NCBIGene:3988': 'True', 'NCBIGen...",No Results: 200,Error: 500,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200{},Results: 200{},Results: 200{},No Results: 200,Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Unknown: 503,Error: 598,Unknown: 503,Results: 200{},Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-bte,No Results: 200,Results: 200{},Results: 200{},Results: 200{},No Results: 200,Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Error: 598,Error: 598,Error: 500,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 500
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 404,Error: 404,Error: 500,No Results: 200,Error: 500,Error: 404,Error: 500
ara-improving,Results: 200{'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200{},Results: 200{},Results: 200{},No Results: 200,No Results: 200,No Results: 200,Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Results: 200{},No Results: 200,No Results: 200,No Results: 200,"Results: 200{'NCBIGene:120892': 'True', 'NCBIG...","Results: 200{'NCBIGene:3988': 'True', 'NCBIGen...",No Results: 200,Results: 200{},No Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,No Results: 200,No Results: 200,Results: 200{},Error: 504,No Results: 200,No Results: 200,No Results: 200,Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Error: 504,Error: 504,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502


<br>

### Highlight the cells In Excel

<br>

In [108]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [109]:
styled = df.style.applymap(highlight)

In [110]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop-related-to,B.2_DILI-three-hop-interacts-with,B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200{},Error: 598,No Results: 200,No Results: 200,Results: 200{},No Results: 200,Results: 200{},Results: 200{},No Results: 200,Results: 200{},Results: 200{},Error: 598,Error: 598,Results: 200{},No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200{},Error: 500,Results: 200{},Results: 200{},Results: 200{},Error: 422,Results: 200{},Error: 598,Error: 500,Results: 200{},No Results: 200,"Results: 200{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}","Results: 200{'NCBIGene:3988': 'True', 'NCBIGene:5627': 'False', 'NCBIGene:7043': 'False'}",No Results: 200,Error: 500,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200{},Results: 200{},Results: 200{},No Results: 200,Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},Unknown: 503,Error: 598,Results: 200{},Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Results: 200{},Results: 200{},Unknown: 503,Unknown: 503,Unknown: 503,Error: 400,Results: 200{},Unknown: 503,Error: 598,Unknown: 503,Results: 200{},Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-bte,No Results: 200,Results: 200{},Results: 200{},Results: 200{},No Results: 200,Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},Error: 598,Error: 598,Results: 200{},Error: 500,Error: 500,Error: 598,No Results: 200,No Results: 200,Error: 598,Results: 200{},Results: 200{},Error: 598,Error: 598,No Results: 200,Results: 200{},Error: 598,Error: 598,Error: 500,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 500
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 404,Error: 404,Error: 500,No Results: 200,Error: 500,Error: 404,Error: 500
ara-improving,Results: 200{'PUBCHEM.COMPOUND:2950270': 'True'},Results:

In [111]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [112]:
wks_name

'Workflow Progress Tracker_2021_09_27-12_00_27_PM'

In [113]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [53]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop-related-to,B.2_DILI-three-hop-interacts-with,B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,Error: 422,Results: 200,Error: 598,Error: 500,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Unknown: 503,Error: 598,Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Results: 200,Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Error: 400,Results: 200,Unknown: 503,Error: 598,Unknown: 503,Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200,Error: 500,Error: 500,Error: 598,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Error: 598,Error: 598,No Results: 200,Results: 200,Error: 598,Error: 598,Error: 500,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 500
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 404,Error: 404,Error: 500,No Results: 200,Error: 500,Error: 404,Error: 500
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Resu

In [54]:
df['Query Type'] = 'Sync'

df = df[['Query Type']+ list(df.columns[:-1])]

In [55]:
#df.reset_index(inplace=True)

In [56]:
#df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [61]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [62]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [63]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_09_27-08_32_27_AM' id:431338910>

### Push Dataframe 2

<br>

In [64]:
wks2 = 'edge_attribute_source_' + date

In [65]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_09_27-08_32_27_AM' id:1091907801>

In [66]:
wks2

'edge_attribute_source_2021_09_27-08_32_27_AM'

In [67]:
wks_name

'Workflow Progress Tracker_2021_09_27-08_32_27_AM'